In [29]:
import os, os.path
import fnmatch
import json
import nltk
import pycountry
#nltk.download('wordnet')
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import wordnet

final_entity_list_map = {}
entity_recognition_map = {}


In [30]:
with open('config.json') as handle:
    config = json.load(handle)
book = config['book_name']
Parts = config['number_of_parts']
dir_path = './%s/part'% book

In [31]:
lang = {}
for i in list(pycountry.languages):
    lang[i.name] = 1
lang['Samskrit'] = 1

In [33]:
with open('json_files/%s/spacy_chapter_entities_map.json' % book) as handle:
    spacy_chapter_entity_map = json.load(handle)

with open('json_files/%s/nltk_chapter_entities_map.json' % book) as handle:
    nltk_chapter_entity_map = json.load(handle)
    
with open('json_files/%s/polygot_chapter_entities_map.json' % book) as handle:
    polygot_chapter_entity_map = json.load(handle)
                
with open('json_files/%s/polygot_entity_list_map.json' % book) as handle:
    polygot_entity_list_map = json.load(handle)

with open('json_files/%s/spacy_entity_list_map.json' % book) as handle:
    spacy_entity_list_map = json.load(handle)

with open('json_files/%s/nltk_entity_list_map.json' % book) as handle:
    nltk_entity_list_map = json.load(handle)
            
with open('json_files/%s/location_list_map.json' % book) as handle:
    location_list_map = json.load(handle)
    
#comment/uncomment as final_person_list is ready    
with open('json_files/%s/final_person_list.json' % book) as handle:
    people_list_map = json.load(handle)

In [34]:
final_chapter_entity_map = {}
wordnets = []
        
        
for p in range(1,Parts+1):
    part = str(p)
    final_chapter_entity_map[part]={}
    entity_recognition_map[part] = {}
    for top_percent in [100]:
        
        x = 1
        print(dir_path + str(part) + '/')
        no_of_chapters = len(fnmatch.filter(os.listdir(dir_path + str(part) + '/'), '*.txt'))
        print('no.of_chapters:' + str(no_of_chapters))
        print('part:' + str(part))
        
        print('chapters' + str(x) + ' to ' + str(no_of_chapters) )
        total_entity_set = set()

        for j in range(x,no_of_chapters+1):
            i = str(j)
            final_chapter_entity_map[part][i] = set(polygot_chapter_entity_map[part][i]) | set(spacy_chapter_entity_map[part][i]) | set(nltk_chapter_entity_map[part][i])
            for e in final_chapter_entity_map[part][i]:
                total_entity_set.add(e)
            final_chapter_entity_map[part][i] = list(final_chapter_entity_map[part][i])

            #entity should be pointed out by atleast 2 libraries
            total_entity_set_copy = set()
            for e in total_entity_set:
                # removing entities which are languages
                if e in lang:
                    continue
                presence_count = 0
                if e in polygot_entity_list_map[part].keys():
                    presence_count += 1
                if e in spacy_entity_list_map[part].keys():
                    presence_count += 1
                if e in nltk_entity_list_map[part].keys():
                    presence_count += 1

                if presence_count < 2:
                    continue
                total_entity_set_copy.add(e)
            total_entity_set = total_entity_set_copy

        #checking if any entity is substring of another entity neglect if true
            total_entity_c = total_entity_set
            total_entity_set_cc = set()
            for e in total_entity_set:
                total_entity_c.remove(e)
                if not any(e in s for s in total_entity_c):
                    total_entity_set_cc.add(e)
                total_entity_c.add(e)
            total_entity_set = total_entity_set_cc

            #print(total_entity_set)
            
            for e in total_entity_set:
                
                final_entity_list_map[e] = []
                if e in polygot_entity_list_map[part].keys():
                    final_entity_list_map[e].extend(polygot_entity_list_map[part][e])
                if e in spacy_entity_list_map[part].keys():
                    final_entity_list_map[e].extend(spacy_entity_list_map[part][e])
                if e in nltk_entity_list_map[part].keys():
                    final_entity_list_map[e].extend(nltk_entity_list_map[part][e])

                entity_type = max(set(final_entity_list_map[e]), key=(final_entity_list_map[e]).count)
                    

                if entity_type==1 and e not in location_list_map[part][i]:
                    #print(e,i)
                    continue
                if entity_type==0 and e in location_list_map[part][i]:
                    continue
                
                
                # person entity needs to be in people list map mind the cyclic dependancy due to manual filtering
                if entity_type == 0 and e not in people_list_map[part]:
                    continue
                if wordnet.synsets(e) and entity_type==0:
                    wordnets.append(e)
                    continue
                entity_recognition_map[part][e] = entity_type
            



./Lenin_Selected_Works/part1/
no.of_chapters:27
part:1
chapters1 to 27


In [35]:
#only keeping the entities that perfectly passed all filters in chapter entity map also
for p in range(1,Parts+1):
    part = str(p)
    x = 1
    print(dir_path + str(part) + '/')
    no_of_chapters = len(fnmatch.filter(os.listdir(dir_path + str(part) + '/'), '*.txt'))
    print('no.of_chapters:' + str(no_of_chapters))
    print('part:' + str(part))

    print('chapters' + str(x) + ' to ' + str(no_of_chapters) )
    entity_removed = 0
    for j in range(x,no_of_chapters+1):
        chapter_filtered_list = []
        j = str(j)
        for entity in final_chapter_entity_map[part][j]:
            if entity in entity_recognition_map[part].keys():
                chapter_filtered_list.append(entity)
        final_chapter_entity_map[part][j] = chapter_filtered_list
print("entity removed:" + str(entity_removed))

./Lenin_Selected_Works/part1/
no.of_chapters:27
part:1
chapters1 to 27
entity removed:0


In [36]:
print(set(wordnets))

with open('json_files/%s/final_chapter_entity_map_tfidf.json' % book, 'w') as handle:
    json.dump(final_chapter_entity_map,handle,indent = 4)

with open('json_files/%s/entity_recognition_map_tfidf.json' % book, 'w') as handle:
    json.dump(entity_recognition_map,handle,indent = 4)

set()


In [37]:
with open('json_files/%s/entity_recognition_map_tfidf.json' % book) as handle:
    entity_map = json.load(handle)
place_list = {}
person_list = {}
for part in range(1,Parts+1):
    part = str(part)
    place_list[part] = set()
    person_list[part] = set()
    for entity in entity_map[part].keys():
        if entity_map[part][entity] == 1:
            place_list[part].add(entity)
        else:
            person_list[part].add(entity)
    person_list[part] = list(person_list[part])
    place_list[part] = list(place_list[part])

In [20]:
# only person list is needed to be filtered manually initial_person_list -> final_person_list
with open('json_files/%s/initial_person_list.json' % book, 'w') as handle:
    json.dump(person_list,handle,indent = 4)

with open('json_files/%s/initial_place_list.json' % book, 'w') as handle:
    json.dump(place_list,handle,indent = 4)